# AstroPy: Handling FITS files

Documentation
-------------

For more information about the features presented below, you can read the
[astropy.io.fits](http://docs.astropy.org/en/stable/io/fits/index.html) docs.


Reading FITS files and accessing data
-------------------------------------

Opening a FITS file is relatively straightforward. Here we open an image from the [IPHAS](http://www.iphas.org) imaging survey:

In [1]:
from astropy.io import fits

In [2]:
hdulist = fits.open('./656nmos.fits')

The returned object, ``hdulist``, (an instance of the [HDUList class](http://docs.astropy.org/en/stable/io/fits/api/hdulists.html#hdulist)) behaves like a Python list, and each element
maps to a Header-Data Unit (HDU) in the FITS file. You can view more
information about the FITS file with:

In [3]:
hdulist.info()

Filename: ./656nmos.fits
No.    Name         Type      Cards   Dimensions   Format
  0  PRIMARY     PrimaryHDU     290   (1600, 1600)   float32   
  1  656nmos_cvt.tab  TableHDU       353   1R x 49C   [D25.17, D25.17, E15.7, E15.7, E15.7, E15.7, E15.7, E15.7, E15.7, E15.7, A1, E15.7, I12, I12, D25.17, D25.17, A8, A8, I12, E15.7, E15.7, E15.7, E15.7, E15.7, E15.7, I12, I12, I12, I12, I12, I12, I12, I12, A48, E15.7, E15.7, E15.7, E15.7, E15.7, E15.7, E15.7, E15.7, E15.7, E15.7, E15.7, E15.7, E15.7, E15.7, E15.7]   


As we can see, this file contains two HDUs. The first contains the image, the second a data table. To access the primary HDU, which contains the main data, you can then do:

In [ ]:
hdu = hdulist[0]

The ``hdu`` object then has two important attributes: ``data``, which behaves
like a Numpy array, can be used to access the data, and ``header``, which
behaves like a dictionary, can be used to access the header information.
First, we can take a look at the data:

In [ ]:
hdu.data.shape

This tells us that it is a 1600-by-1600 pixel image. We can now take a peak at the header:

In [ ]:
hdu.header

We can access individual header keywords using standard item notation:

In [ ]:
hdu.header['INSTRUME']

In [ ]:
hdu.header['EXPTIME']

We can plot the image using matplotlib:

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(np.log10(hdu.data), origin='lower', cmap='gray', vmin=1.5, vmax=3)

Note that this is just a plot of an array, so the coordinates are just pixel
coordinates at this stage.

Modifying data or header information in a FITS file object is easy. We can
update header keywords:

In [ ]:
hdu.header['INSTRUME'] = "Wide-Field Planetary Camera 2 (WFPC2)"

or add new ones:

In [ ]:
hdu.header['MODIFIED'] = '2014-12-01'  # adds a new keyword

and we can also change the data, for example subtracting a background value:

In [ ]:
hdu.data = hdu.data - 0.5

This only changes the FITS file in memory. You can write to a file with:

In [ ]:
hdu.writeto('hubble-image-background-subtracted.fits', clobber=True)

Creating a FITS file from scratch
---------------------------------

If you want to create a FITS file from scratch, you need to start off by creating an HDU object:

In [ ]:
hdu = fits.PrimaryHDU()

and you can then populate the data and header attributes with whatever information you like:

In [ ]:
import numpy as np

In [ ]:
hdu.data = np.random.random((128, 128))

Note that setting the data automatically populates the header with basic information:

In [ ]:
hdu.header

and you should never have to set header keywords such as ``NAXIS``, ``NAXIS1``, and so on manually. We can then set additional header keywords:

In [ ]:
hdu.header['telescop'] = 'Python Observatory'  # Note the keyword is case-insensitive; it will be written all caps

and we can then write out the FITS file to disk:

In [ ]:
hdu.writeto('random_array.fits', clobber=True)

Creating a multi-extension FITS file
------------------------------------

Many observatories format their FITS files such that no data is stored in the primary HDU.  Instead, multiple image extensions are used to store each of the main image data, the data quality array, the error values, etc.  The primary HDU stores no data, but does use the primary header to store metadata common to the observation.

As mentioned before, the `HDUList` object is similar to a Python list, and can be manipulated like one to store multiple HDUs as well as reorder them in a FITS file.  For example we can create a primary HDU containing only metadata:

In [ ]:
pri_hdu = fits.PrimaryHDU()
pri_hdu.header['telescop'] = 'Python Observatory'

and then create an image extension HDU using the `ImageHDU` class:

In [ ]:
img_hdu = fits.ImageHDU(data=np.random.random((128, 128)))

Finally, add both HDUs to an `HDUList` and write it to disk:

In [ ]:
hdul = fits.HDUList([pri_hdu, img_hdu])
hdul.writeto('random_array2.fits', clobber=True)

We can check that the output file is in the format we expected with the `fits.info()` convenience function.  This is a shortcut for the pattern we saw earlier of `hdul = fits.open(...); hdul.info()`:

In [ ]:
fits.info('random_array2.fits')

Because an `HDUList` is like a Python list, new HDUs can also be appended or inserted:

In [ ]:
new_hdu = fits.ImageHDU(data=np.random.random((256, 256)))
hdul.append(new_hdu)
hdul.writeto('random_array3.fits', clobber=True)

This writes a new FITS file similar to the previous one, but with the new HDU inserted:

In [ ]:
fits.info('random_array3.fits')

Convenience functions
---------------------

Like `fits.info`, a few other shortcut "convenience" functions are provided.  For example, in cases where you just want to access the data or header in a specific HDU, you can use the following convenience functions:

In [ ]:
data = fits.getdata('http://star.herts.ac.uk/~gb/python/656nmos.fits')
header = fits.getheader('http://star.herts.ac.uk/~gb/python/656nmos.fits')

To get the data or header for an HDU other than the first, you can specify the
extension name or index:

In [ ]:
data = fits.getdata("http://star.herts.ac.uk/~gb/python/656nmos.fits", ext=1)

and similarly for ``getheader``.  The documentation provides [a list](http://docs.astropy.org/en/stable/io/fits/api/files.html#writeto) of other such functions.